In [ ]:
# Select what type of street network you want to load
network_filter = osm.osmium_network_filter("all")
# Download the region pbf file crop it by aoi and convert to osm format
osm.geofabrik_to_osm(
    osm_xml_file,
    input_file=results_path,
    aoi=aoi,
    osmium_filter_args=network_filter,
    overwrite=False
)

In [ ]:
min_edge_length = reference_distance/10

if os.path.isfile(streets_path):
    street_edges = gpd.GeoDataFrame(streets_path)
    G = ox.load_graphml(streets_graph_path)
else:
    # Load
    G = ox.graph_from_xml(osm_xml_file)
    # Project geometry coordinates to UTM system to allow euclidean meassurements in meters (sorry americans)
    G = ox.project_graph(G,to_crs=aoi.estimate_utm_crs())
    # Save the graph in graphml format to avoid the slow loading process
    G = graph_processing.simplify_graph(G,min_edge_length=min_edge_length,min_edge_separation=min_edge_length*2,undirected=True)
    # Save the result in graphml format
    ox.save_graphml(G,streets_graph_path)

    street_edges = ox.graph_to_gdfs(G,nodes=False)
    street_edges = street_edges.to_crs(aoi.crs)
    street_edges.to_file(streets_path)